In [14]:
import regex as re
import pandas as pd
import os
import random
import numpy as np
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='your module name')

In [15]:
data = pd.read_csv('../nlpia/topic_analysis/smalldata_washed.csv')
rumor = data['rumor'].to_list()
reverse = data['reverse'].to_list()
rumor_class = len(rumor)*[1]
reverse_class = len(reverse)*[0]
data = rumor + reverse
data_class = rumor_class + reverse_class
#后续添加数据要放在raw目录下，使用.xlsx格式将后续标注的数据加入data
raw_data_list = os.listdir('../nlpia/topic_analysis/raw')
data_list = []
good_name = re.compile(r'^(?!(\~\$)).*(.xlsx)')
for i in raw_data_list:
    if good_name.match(i):
    #    data_list.append(i)
        temp = pd.read_excel('../nlpia/topic_analysis/raw/'+i)
        temp.fillna('',inplace=True)
        temp_rumor = [x.strip() for x in  temp['punc_rumor'].to_list()  if x.strip()!='']
        temp_reverse = [x.strip() for x in  temp['punc_truth'].to_list()  if x.strip()!='']
        temp_rumor_class = len(temp_rumor)*[1]
        temp_reverse_class = len(temp_reverse)*[0]
        temp_data = temp_rumor + temp_reverse
        temp_data_class = temp_rumor_class + temp_reverse_class
        for i in range( len(temp_data)):
            temp_data[i]=re.sub(r'[\,\'\ ]|(\n)','',temp_data[i])
    data = data + temp_data
    data_class = data_class + temp_data_class
with open('../nlpia/topic_analysis/data.txt','w') as f:
    for i in range(len(data)):
        f.write(data[i]+'\t'+str(data_class[i])+'\n')
        data_list =list( zip(data, data_class))
random.shuffle(data_list)
data, data_class = zip(*data_list)
index = ['rumor{}{}'.format(i, '!' * j) for (i, j) in zip(range(len(data)), data_class)]
df = pd .DataFrame(data=zip(data_class,data), columns=["rumor","text"], index=index)

# 去除停用词
with open ('../nlpia/topic_analysis/hit_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# jieba分词
import jieba
for i in range(len(df)):
    words = jieba.cut(df["text"].iloc[i],cut_all=False)
    words = [word for word in words if word not in stopwords]
    df["text"].iloc[i] = ' '.join(words) 

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=df.text)
                        .toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),
                                     counter.vocabulary_.keys())))
bow_docs.columns = terms

try:
    print('rumor0:'+df.loc['rumor0'].text)
    print(bow_docs.loc['rumor0'][bow_docs.loc['rumor0'] > 0].head())
except KeyError:
    print('rumor0!:'+df.loc['rumor0!'].text)
    print(bow_docs.loc['rumor0!'][bow_docs.loc['rumor0!'] > 0].head())

/var/folders/f0/6j6j3w814n53skbt2s4d8p5m0000gn/T/ipykernel_61938/1208280586.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"].iloc[i] = ' '.join(words)


#### 对比TF-IDF向量的分类

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize,)
tfidf_docs =tfidf.fit_transform(raw_documents=df.text).toarray()
tfidf_docs = tfidf_docs - tfidf_docs.mean(axis=0)

/Users/stevenyu/opt/anaconda3/envs/d2l/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [19]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(tfidf_docs,df.rumor,test_size=0.2,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA()
lda = lda.fit(X_train_res, y_train_res)
df['tfidf_rumor'] = lda.predict(tfidf_docs)

过采样前训练集中类别0的数量： 1505
过采样前训练集中类别1的数量： 722
过采样后训练集中类别0的数量： 1505
过采样后训练集中类别1的数量： 1505


In [20]:
round(float(lda.score(X_test, y_test)), 3)

0.801

In [21]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from tensorboardX import SummaryWriter

y_true = df.loc[y_test.index].rumor
y_pred = df.loc[y_test.index].tfidf_rumor
loss_total = 0
acc = metrics.accuracy_score(y_true, y_pred)
report = metrics.classification_report(y_true, y_pred, target_names=["non-rumor","rumor"], digits=4)
confusion = metrics.confusion_matrix(y_true, y_pred)
print(report)
print(confusion)
with open("../Chinese-Text-Classification-Pytorch/report/"+'tfidf'+'.txt', 'w', encoding='utf-8') as f:
    print(report,file=f)

with open("../Chinese-Text-Classification-Pytorch/report_matrix/"+'tfidf'+'_matrix'+'.txt', 'w', encoding='utf-8') as f:
    print(confusion,file=f)
np.savez("../Chinese-Text-Classification-Pytorch/report_matrix/"+'tfidf'+'_matrix'+'.npz', matrix= confusion)    

              precision    recall  f1-score   support

   non-rumor     0.8947    0.7896    0.8389       366
       rumor     0.6709    0.8220    0.7388       191

    accuracy                         0.8007       557
   macro avg     0.7828    0.8058    0.7889       557
weighted avg     0.8180    0.8007    0.8046       557

[[289  77]
 [ 34 157]]


#### 隐形狄利克雷分布原理
- 设想一台自动生成文档的机器，只有两个选项来控制生成文档的两个属性
    - 生成文档的词的数量（泊松分布）
    - 文档中混合的主题的数量（狄利克雷分布）
- 除此之外，最关键的问题在于确定一个主题-词项矩阵，该矩阵表示了每个词对主题的贡献权重。一但能够确定这个矩阵，机器就可以在选择好的主题上反复迭代选择词，直到生成一篇足够长度的文档。
- 回到对现有文档估计主题的问题上来，LDiA可以用于关于词和主题的参数，Blei和Ng通过分析语料库中文档的统计数据确定两个参数
    - 第一个参数可以通过计算语料库的平均词（n-gram）数量、
    - 第二个参数更棘手，必须先猜测有几个主题数再为主题分配语词，最后优化目标函数
-  “LSA试图将原本分散的东西分散开，LDiA试图将原本接近的东西接近在一起”
LDiA使用100个主题

In [22]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia100 = LDiA(n_components=100, learning_method='batch')
ldia100 = ldia100.fit(bow_docs)
ldia100.components_.shape

(100, 9200)

In [23]:
ldia100_topic_vectors = ldia100.transform(bow_docs)
columns100 = ['topic{}'.format(i) for i in range(ldia100.components_.shape[0])]
ldia100_topic_vectors = pd.DataFrame(ldia100_topic_vectors,index=index, columns=columns100)
ldia100_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic90,topic91,topic92,topic93,topic94,topic95,topic96,topic97,topic98,topic99
rumor0!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rumor1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rumor2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rumor3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rumor4!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(ldia100_topic_vectors,df.rumor,test_size=0.2,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA()
lda = lda.fit(X_train_res, y_train_res)
df['ldia100_rumor'] = lda.predict(ldia100_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

过采样前训练集中类别0的数量： 1505
过采样前训练集中类别1的数量： 722
过采样后训练集中类别0的数量： 1505
过采样后训练集中类别1的数量： 1505


0.56

In [25]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from tensorboardX import SummaryWriter

y_true = df.loc[y_test.index].rumor
y_pred = df.loc[y_test.index].ldia100_rumor
loss_total = 0
acc = metrics.accuracy_score(y_true, y_pred)
report = metrics.classification_report(y_true, y_pred, target_names=["non-rumor","rumor"], digits=4)
confusion = metrics.confusion_matrix(y_true, y_pred)
print(report)
print(confusion)
with open("../Chinese-Text-Classification-Pytorch/report/"+"ldia"+'.txt', 'w', encoding='utf-8') as f:
    print(report,file=f)
with open("../Chinese-Text-Classification-Pytorch/report_matrix/"+'ldia'+'_matrix'+'.txt', 'w', encoding='utf-8') as f:
    print(confusion,file=f)
np.savez("../Chinese-Text-Classification-Pytorch/report_matrix/"+'ldia'+'_matrix'+'.npz', matrix= confusion)

              precision    recall  f1-score   support

   non-rumor     0.7051    0.5683    0.6293       366
       rumor     0.3969    0.5445    0.4592       191

    accuracy                         0.5601       557
   macro avg     0.5510    0.5564    0.5443       557
weighted avg     0.5994    0.5601    0.5710       557

[[208 158]
 [ 87 104]]


# PCA

In [26]:
from sklearn.decomposition import PCA
k=min(len(tfidf.vocabulary_),len(df))
k=100
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
base = sum(pca.singular_values_)


In [27]:
nums = pca.singular_values_.tolist()

In [28]:
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=index)
pca_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic90,topic91,topic92,topic93,topic94,topic95,topic96,topic97,topic98,topic99
rumor0!,0.024,-0.036,-0.030,0.014,-0.036,-0.048,-0.001,0.002,-0.031,-0.000,...,-0.001,-0.013,-0.008,0.013,0.008,-0.001,0.014,-0.010,0.000,-0.014
rumor1,-0.031,-0.052,0.026,0.037,-0.004,-0.023,-0.040,-0.047,0.023,0.029,...,-0.023,0.087,-0.044,-0.079,-0.013,-0.052,-0.023,-0.026,0.042,-0.020
rumor2,0.092,0.110,0.104,0.010,0.048,0.031,-0.036,-0.020,0.003,0.035,...,0.028,0.026,-0.058,0.022,-0.013,0.012,0.032,0.025,0.040,0.042
rumor3,0.033,-0.028,-0.053,0.018,0.097,-0.103,0.039,0.072,0.021,-0.017,...,-0.025,0.010,0.034,-0.020,-0.037,0.054,0.080,-0.020,-0.093,-0.095
rumor4!,0.024,-0.040,-0.038,0.023,-0.037,-0.058,0.001,0.016,-0.054,-0.005,...,-0.000,-0.025,-0.068,0.006,-0.001,0.056,-0.018,0.112,-0.058,-0.022
rumor5,0.011,-0.023,-0.031,0.045,-0.026,-0.027,-0.040,0.026,-0.010,-0.047,...,0.081,0.018,0.014,0.044,-0.004,-0.026,0.075,0.033,0.017,-0.015


In [29]:
tfidf.vocabulary_
#根据词项的频率对词汇表进行排序
#当对某个不按照最左边元素排序的序列解压并在排序后重新压缩时，可以使用zip(*sorted(zip(...)))
column_nums , terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))


In [30]:
weights = pd.DataFrame(pca.components_, columns=terms,
                       index = ['topic{}'.format(i) for i in range(pca.components_.shape[0])])
pd.options.display.max_columns = 8
weights.head(4).round(3)

,%,.,0,0t,...,鼻炎,鼻翼,龋齿,龙葵
topic0,0.001,-0.001,0.0,-0.000,...,-0.000,0.000,0.001,-0.001
topic1,-0.001,0.000,-0.0,-0.001,...,-0.002,-0.000,-0.001,-0.002
topic2,-0.001,0.001,-0.0,-0.001,...,-0.002,-0.001,-0.001,0.000
topic3,0.000,0.004,0.0,0.001,...,-0.002,0.001,0.001,-0.002


In [31]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=k,n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf_docs)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=index)
svd_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,...,topic96,topic97,topic98,topic99
rumor0!,0.024,-0.036,-0.030,0.014,...,-0.005,-0.008,0.019,0.026
rumor1,-0.031,-0.052,0.026,0.037,...,-0.030,-0.026,0.041,-0.047
rumor2,0.092,0.110,0.104,0.010,...,0.004,-0.028,-0.001,0.018
rumor3,0.033,-0.028,-0.053,0.018,...,-0.036,0.026,-0.047,0.051
rumor4!,0.024,-0.040,-0.038,0.023,...,-0.087,-0.004,-0.072,0.021
rumor5,0.011,-0.023,-0.031,0.045,...,0.009,0.049,0.023,0.051


In [32]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(svd_topic_vectors,df.rumor,test_size=0.2 ,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA(n_components=1)
lda = lda.fit(X_train_res, y_train_res)
df['LSA16_rumor'] = lda.predict(svd_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

过采样前训练集中类别0的数量： 1505
过采样前训练集中类别1的数量： 722
过采样后训练集中类别0的数量： 1505
过采样后训练集中类别1的数量： 1505


0.758

In [33]:
import numpy as np
svd_topic_vectors = (svd_topic_vectors.T / np.linalg.norm(svd_topic_vectors,axis=1)).T
svd_topic_vectors.iloc[:10].dot(svd_topic_vectors.iloc[:10].T).round(1)

,rumor0!,rumor1,rumor2,rumor3,...,rumor6,rumor7!,rumor8,rumor9!
rumor0!,1.0,0.1,-0.1,0.1,...,0.1,0.2,-0.1,-0.1
rumor1,0.1,1.0,-0.1,-0.0,...,0.0,-0.1,0.2,0.0
rumor2,-0.1,-0.1,1.0,-0.1,...,0.0,-0.1,-0.0,-0.0
rumor3,0.1,-0.0,-0.1,1.0,...,0.0,0.1,-0.0,-0.1
rumor4!,0.2,0.0,-0.0,-0.1,...,-0.1,0.0,-0.1,-0.0
rumor5,0.2,0.0,-0.0,0.0,...,-0.1,0.1,0.1,-0.1
rumor6,0.1,0.0,0.0,0.0,...,1.0,0.1,-0.1,0.1
rumor7!,0.2,-0.1,-0.1,0.1,...,0.1,1.0,-0.0,0.1
rumor8,-0.1,0.2,-0.0,-0.0,...,-0.1,-0.0,1.0,-0.1
rumor9!,-0.1,0.0,-0.0,-0.1,...,0.1,0.1,-0.1,1.0


In [34]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lda, svd_topic_vectors, df.rumor, cv=5)
print("rumor Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))


rumor Accuracy: 0.7877 (+/- 0.0204)


In [35]:
from sklearn.metrics import f1_score, precision_recall_fscore_support
y_true = df.rumor
y_pred = df.LSA16_rumor
p_macro, r_macro, f_macro, support_macro \
    = precision_recall_fscore_support(y_true=y_true, y_pred=y_pred, labels=[0,1], average='macro')

p_micro, r_micro, f_micro, support_micro\
    = precision_recall_fscore_support(y_true=y_true, y_pred=y_pred, labels=[0,1], average='micro')

print('macro: p {}, r {}, f1 {}'.format(p_macro, r_macro, f_macro))

print('micro: p {}, r {}, f1 {}'.format(p_micro, r_micro, f_micro))

macro: p 0.7759722714561424, r 0.8076293317675737, f1 0.7814652675478351
micro: p 0.7938218390804598, r 0.7938218390804598, f1 0.7938218390804598


In [36]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from tensorboardX import SummaryWriter

y_true = df.loc[y_test.index].rumor
y_pred = df.loc[y_test.index].LSA16_rumor
loss_total = 0
acc = metrics.accuracy_score(y_true, y_pred)
report = metrics.classification_report(y_true, y_pred, target_names=["non-rumor","rumor"], digits=4)
confusion = metrics.confusion_matrix(y_true, y_pred)
print(report)
with open("../Chinese-Text-Classification-Pytorch/report/"+'pca'+'.txt', 'w', encoding='utf-8') as f:
    print(report,file=f)
print(confusion)
with open("../Chinese-Text-Classification-Pytorch/report_matrix/"+'pca'+'_matrix'+'.txt', 'w', encoding='utf-8') as f:
    print(confusion,file=f)
np.savez("../Chinese-Text-Classification-Pytorch/report_matrix/"+'pca'+'_matrix'+'.npz', matrix= confusion)    

              precision    recall  f1-score   support

   non-rumor     0.8812    0.7295    0.7982       366
       rumor     0.6102    0.8115    0.6966       191

    accuracy                         0.7576       557
   macro avg     0.7457    0.7705    0.7474       557
weighted avg     0.7883    0.7576    0.7634       557

[[267  99]
 [ 36 155]]
